In [36]:
import pandas as pd
from Utils.DatasetClass import SmileDataset
import torchvision
from glob import glob
from torch.utils.data import Dataset
from torch.utils.data import DataLoader

import torch
import torch.nn as nn
import numpy as np
import pandas as pd

## Prepping test datatset

In [38]:
test_file_path = "./modified-combined-public.csv"
test_images_path = "./test-public-faces/"

all_images = glob(test_images_path + "*/*/*.jpg")
test_person_to_images = {}

test_images = []

for x in all_images:

    if x.split("\\")[-3] + "/" + x.split("\\")[-2] not in test_person_to_images:
        test_person_to_images[x.split("\\")[-3] + "/" + x.split("\\")[-2]] = [x]

    else:
        test_person_to_images[x.split("\\")[-3] + "/" + x.split("\\")[-2]].append(x)

    test_images.append(x)

test_people = [x.split("\\")[-3] + "/" + x.split("\\")[-2] for x in test_images]
test_people = list(dict.fromkeys(test_people)) # removing the duplicates

relationships = pd.read_csv(test_file_path)
relationships = list(zip(relationships.p1.values, relationships.p2.values))
print(relationships)

test_relationships = [x for x in relationships if x[0] in test_people and x[1] in test_people] #Check if people are in the testing dataset

test_dataset = SmileDataset(relations = test_relationships, person_to_image= test_person_to_images)
testloader = DataLoader(test_dataset, batch_size= 1, shuffle = True)

[('F0001/MID3', 'F0001/MID4'), ('F0007/MID1', 'F0007/MID8'), ('F0008/MID1', 'F0008/MID4'), ('F0008/MID8', 'F0008/MID10'), ('F0016/MID1', 'F0016/MID2'), ('F0021/MID1', 'F0021/MID4'), ('F0028/MID3', 'F0028/MID4'), ('F0030/MID1', 'F0030/MID4'), ('F0035/MID2', 'F0035/MID3'), ('F0036/MID1', 'F0036/MID4'), ('F0036/MID1', 'F0036/MID7'), ('F0038/MID5', 'F0038/MID6'), ('F0045/MID3', 'F0045/MID4'), ('F0046/MID4', 'F0046/MID5'), ('F0047/MID2', 'F0047/MID3'), ('F0048/MID1', 'F0048/MID2'), ('F0048/MID1', 'F0048/MID3'), ('F0048/MID1', 'F0048/MID4'), ('F0048/MID2', 'F0048/MID4'), ('F0048/MID2', 'F0048/MID3'), ('F0048/MID3', 'F0048/MID4'), ('F0053/MID3', 'F0053/MID5'), ('F0054/MID3', 'F0054/MID4'), ('F0060/MID3', 'F0060/MID4'), ('F0064/MID3', 'F0064/MID4'), ('F0064/MID3', 'F0064/MID5'), ('F0064/MID4', 'F0064/MID5'), ('F0067/MID3', 'F0067/MID4'), ('F0069/MID2', 'F0069/MID6'), ('F0071/MID3', 'F0071/MID4'), ('F0077/MID4', 'F0077/MID6'), ('F0080/MID6', 'F0080/MID8'), ('F0082/MID4', 'F0082/MID5'), ('F0083/

## Test function definition

In [35]:
def testing(model, testloader, test_dataset, device):
    model.eval()
    model.to(device)
    running_corrects = 0
    
    for batch in testloader:
        tensor1, tensor2, label = batch
        tensor1, tensor2, label = tensor1.to(device), tensor2.to(device), label.float().view(-1,1).to(device)
        with torch.no_grad():
            output = model(tensor1, tensor2)
            preds = output>0.5
            
        running_corrects += torch.sum(preds == (label>0.5))
    
    test_acc = running_corrects.item()/len(test_dataset)

    return test_acc

## Instantiating Model

In [32]:
from Utils.SiameseNet import SiameseNet
import os

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model_choice = 'test_epoch100.pt'
MODEL_PATH = os.getcwd() + '\\models\\'+model_choice

model = SiameseNet()
model.load_state_dict(torch.load(MODEL_PATH, map_location= device))
model.eval()

SiameseNet(
  (encoder): InceptionResnetV1(
    (conv2d_1a): BasicConv2d(
      (conv): Conv2d(3, 32, kernel_size=(3, 3), stride=(2, 2), bias=False)
      (bn): BatchNorm2d(32, eps=0.001, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU()
    )
    (conv2d_2a): BasicConv2d(
      (conv): Conv2d(32, 32, kernel_size=(3, 3), stride=(1, 1), bias=False)
      (bn): BatchNorm2d(32, eps=0.001, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU()
    )
    (conv2d_2b): BasicConv2d(
      (conv): Conv2d(32, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn): BatchNorm2d(64, eps=0.001, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU()
    )
    (maxpool_3a): MaxPool2d(kernel_size=3, stride=2, padding=0, dilation=1, ceil_mode=False)
    (conv2d_3b): BasicConv2d(
      (conv): Conv2d(64, 80, kernel_size=(1, 1), stride=(1, 1), bias=False)
      (bn): BatchNorm2d(80, eps=0.001, momentum=0.1, affine=True, t

In [33]:
test_acc = testing(model, testloader, test_dataset, device)

In [34]:
print("Test Accuracy : {}".format(test_acc))

Test Accuracy : (0.0, 0.7386133996580133)
